In [9]:
from keras.models import load_model
from collections import deque   
import numpy as np
import time
import pickle
import cv2
from PIL import Image as im 
import json
from urllib.request import urlopen
import firebase_admin 
from email.mime.base import MIMEBase
from email import encoders
from firebase_admin import credentials,db,storage, messaging

In [11]:
# idhr appne system ke according jo mai model bhejunga uska path and pickle ka bhi
# model = load_model(r'C:\Users\hp\Downloads\data\video_model\model') 
model = load_model(r"C:\Users\hp\Desktop\final models rj police\model 1 mera wala\model") 
# lb = pickle.loads(open(r"C:\Users\hp\Downloads\data\video_model\labelbinarizer.pickle", "rb").read())
lb = pickle.loads(open(r"C:\Users\hp\Desktop\final models rj police\model 1 mera wala\labelbinarizer.pickle", "rb").read())
# print(pickle.loads(open(r"C:\Users\hp\Desktop\final models rj police\model 1 mera wala\video_model\labelbinarizer.pickle"))
mean = np.array([123.68, 116.779, 103.939][::1], dtype="float32")
Queue = deque(maxlen=128)
import smtplib
import os
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage

smtp_port = 587
smtp_server = "smtp.gmail.com"

cred = credentials.Certificate("credentials.json")
# firebase_admin.initialize_app(cred,{"storageBucket":"rajasthanpolice-bc516.appspot.com"})
bucket = storage.bucket()
registration_token = 'ctSdcSSDSZOwPqBYoy-nBJ:APA91bHEpljSq8xxAJiTtD2IvTQS9fjhb8OvoCpFNCK6n7ljiEhpbxQRUvHNoKeK5MEgvPHWICE7iyvk8it4Uu2joxu01uT4HwyFBbkBqXqCgkEfciJ2o_kSm2FkBUMVfU8af3YOz3OE'

In [12]:
# anomaly_label = ['Arson','Assault','Burglary','Explosion','Fighting','Normal','Road_Accident','Robbery','Shooting','Vandalism']
anomaly_label = ['Arson','Fighting_Assault','Normal','Road_Accident','Robbery_Burglary','Shooting','Vandalism']
# anomaly_label = ['true','false']
# anomaly_label = ['Abuse','Arrest','Arson','Assault','Burglary','Explosion','Fighting',"Normal",'RoadAccidents','Robbery','Shooting','Shoplifting','Stealing','Vandalism']
# 7pne hisab se video ka path dedena wrna 0 for webcamac
output_video_path = r"C:\Users\hp\Downloads\output.avi"  # Specify the desired output video path
fourcc = cv2.VideoWriter_fourcc(*"MJPG")
output_video_writer = None
clip_duration = 35
frame_rate = 30
writer = None
widht,height = None,None

capture_video = cv2.VideoCapture(r"C:\Users\hp\Downloads\Anomaly-Videos-Part-1\Anomaly-Videos-Part-1\Arson\Arson012_x264.mp4") 

while True:
    taken,frame = capture_video.read()  
    if not taken:
        break
    if widht is None or height is None:
        height,widht = frame.shape[:2]  
    output = frame.copy()
    frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame,(244,244)).astype("float16")
#     frame = frame.reshape(244, 244, 3) / 255
    frame -= mean
    preds = model.predict(np.expand_dims(frame,axis=0))[0]
    print(preds)
    Queue.append(preds)
    results = np.array(Queue).mean(axis=0)
    i = np.argmax(results)
#     i = (preds > 0.90)[0] #np.argmax(results)
    label = anomaly_label[i]
    print(results)
#     i = np.argmax(results)
    print(i)
#     label = i
    if(label=="Arson"):
        print("video rec start")
        if output_video_writer is None:
            output_video_writer = cv2.VideoWriter(output_video_path, fourcc, frame_rate, (widht, height), True)
            start_time = time.time()

        output_video_writer.write(output)
        current_time = time.time()
        elapsed_time = current_time - start_time
        if elapsed_time >= clip_duration:
             # Release the VideoWriter and reset variables
            output_video_writer.release()
            output_video_writer = None
            print(f"Video clip saved: {output_video_path}")
     
            data = im.fromarray(output) 
            data.save('arsondummy.jpg') 

            url = "http://ipinfo.io/json"
            response = urlopen(url)
            data = json.load(response)
            print(data)
            video_file_path = "C:/Users/hp/Downloads/output.avi"  # Replace with the actual path to your video
            message = "{data}".format(data=data)
            
#             image =  open(r"C:\Users\hp\Downloads\output.avi", 'rb').read() 
            email_from = "kshitizagarwal1710@gmail.com"
            email_to = "kshitizagarwal1710@gmail.com"
            psw = "zzlpuafbnfcqbvfy"
            msg = MIMEMultipart()
            msg['Subject'] = '{label} detected'.format(label = label)
            msg['From'] = 'kshitizagarwal1710@gmail.com'
            msg['To'] = 'kshitizagarwal1710@gmail.com'
            text = MIMEText(message)
            msg.attach(text)
            with open(video_file_path, "rb") as attachment:
                part = MIMEBase('application', 'octet-stream')
                part.set_payload(attachment.read())
                encoders.encode_base64(part)
                part.add_header('Content-Disposition', f"attachment; filename= {video_file_path.split('/')[-1]}")
                msg.attach(part)

#             image = MIMEImage(image)
#             msg.attach(image)

            txt = msg.as_string()

            TIE = smtplib.SMTP(smtp_server, smtp_port) 
            TIE.starttls()
            TIE.login(email_from, psw)
            print("login success")

            print("sending mail to{email_to}")
            TIE.sendmail(email_from, email_to, txt)
            print("mail sent")
            TIE.quit()
            print("Sending video to cloud")
            blob = bucket.blob("arson.avi")
            blob.upload_from_filename(r"C:\Users\hp\Downloads\output.avi")
            print("video sent to cloud")

            break

    print(label)
    text = label
    cv2.putText(output, text, (35, 50), cv2.FONT_HERSHEY_SIMPLEX,1.25, (255,0,0), 5)

#     if writer is None:
#         fourcc = cv2.VideoWriter_fourcc(*"MJPG")
#         writer = cv2.VideoWriter(r"C:\Users\hp\Downloads\data\video_model\output\output.avi", fourcc, 30,(widht, height), True)
#         writer.write(output)
    cv2.imshow("Output", output)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
print("Cleaning up...")
# writer.release()
capture_video.release()

1/1 [==============================] - 2s 2s/step
[0.19359723 0.13083047 0.14185971 0.1217081  0.32829505 0.03734917
 0.04636022]
[0.19359723 0.13083047 0.14185971 0.1217081  0.32829505 0.03734917
 0.04636022]
4
Robbery_Burglary
1/1 [==============================] - 0s 155ms/step
[0.190425   0.12853105 0.14384694 0.12361461 0.3303384  0.03720921
 0.04603479]
[0.19201112 0.12968075 0.14285332 0.12266135 0.32931674 0.03727919
 0.0461975 ]
4
Robbery_Burglary
1/1 [==============================] - 0s 143ms/step
[0.19048399 0.1294146  0.14592782 0.12242627 0.3278389  0.03714597
 0.04676243]
[0.19150208 0.12959205 0.14387815 0.12258299 0.32882413 0.03723478
 0.04638581]
4
Robbery_Burglary
1/1 [==============================] - 0s 142ms/step
[0.19006138 0.1305404  0.1471201  0.11860171 0.32719764 0.0386842
 0.04779458]
[0.1911419  0.12982914 0.14468864 0.12158767 0.3284175  0.03759714
 0.04673801]
4
Robbery_Burglary
1/1 [==============================] - 0s 177ms/step
[0.18685709 0.135465   

1/1 [==============================] - 0s 104ms/step
[0.21137479 0.12865072 0.13420278 0.14342698 0.2949185  0.04100346
 0.04642271]
[0.19198857 0.13529564 0.14482301 0.13742432 0.3027547  0.04226032
 0.04545347]
4
Robbery_Burglary
1/1 [==============================] - 0s 105ms/step
[0.20766191 0.13465495 0.13871267 0.15216716 0.28335166 0.04140636
 0.04204524]
[0.19240104 0.13527879 0.14466222 0.1378123  0.3022441  0.04223785
 0.04536378]
4
Robbery_Burglary
1/1 [==============================] - 0s 99ms/step
[0.2085816  0.13446136 0.13832553 0.15286906 0.28237057 0.04132868
 0.04206318]
[0.19281591 0.13525783 0.14449975 0.13819838 0.30173454 0.04221454
 0.04527915]
4
Robbery_Burglary
1/1 [==============================] - 0s 102ms/step
[0.20376733 0.13522169 0.13302405 0.16107495 0.28212693 0.04276448
 0.04202063]
[0.1930897  0.13525692 0.14421286 0.1387703  0.30124435 0.04222829
 0.04519769]
4
Robbery_Burglary
1/1 [==============================] - 0s 100ms/step
[0.20441312 0.135342

1/1 [==============================] - 0s 196ms/step
[0.22531964 0.14528061 0.12063747 0.1436405  0.28077826 0.03697702
 0.04736653]
[0.20929003 0.13655551 0.13048014 0.13850299 0.2992251  0.04173035
 0.044216  ]
4
Robbery_Burglary
1/1 [==============================] - 0s 142ms/step
[0.22555697 0.14487028 0.1217292  0.14395997 0.2794527  0.03711004
 0.04732082]
[0.20950986 0.13666788 0.13036188 0.13857673 0.2989579  0.04166791
 0.04425796]
4
Robbery_Burglary
1/1 [==============================] - 0s 128ms/step
[0.22523564 0.14505419 0.12189507 0.14417541 0.27922255 0.03711636
 0.04730082]
[0.20971954 0.13677968 0.13024898 0.13865137 0.29869476 0.04160722
 0.04429853]
4
Robbery_Burglary
1/1 [==============================] - 0s 131ms/step
[0.21417214 0.15256767 0.13652225 0.13803583 0.27752745 0.03601273
 0.04516194]
[0.20977813 0.13698742 0.13033153 0.13864328 0.29841623 0.04153361
 0.04430989]
4
Robbery_Burglary
1/1 [==============================] - 0s 111ms/step
[0.21351223 0.15514

1/1 [==============================] - 0s 169ms/step
[0.19124185 0.14499648 0.11213244 0.15464066 0.30309796 0.04155253
 0.05233812]
[0.20681873 0.14076547 0.12835425 0.13864842 0.298852   0.04124833
 0.04531283]
4
Robbery_Burglary
1/1 [==============================] - 0s 182ms/step
[0.20141707 0.14241122 0.10505337 0.1455003  0.31547108 0.04033602
 0.04981095]
[0.20676963 0.14078043 0.12814242 0.1387107  0.2990031  0.04124004
 0.04535371]
4
Robbery_Burglary
1/1 [==============================] - 0s 139ms/step
[0.20150052 0.14243512 0.10495518 0.14546734 0.31540412 0.04038227
 0.04985543]
[0.20672216 0.14079535 0.12793352 0.13877158 0.29915082 0.04123231
 0.04539427]
4
Robbery_Burglary
1/1 [==============================] - 0s 133ms/step
[0.20254344 0.14380157 0.10419668 0.14873047 0.31002656 0.04020694
 0.05049437]
[0.20668484 0.14082219 0.12772158 0.1388605  0.29924795 0.04122316
 0.04543981]
4
Robbery_Burglary
1/1 [==============================] - 0s 109ms/step
[0.20635818 0.13483

1/1 [==============================] - 0s 148ms/step
[0.1967581  0.14060512 0.12189611 0.1424935  0.312322   0.03674594
 0.04917922]
[0.20799446 0.14044599 0.12607032 0.13980955 0.30041847 0.03963901
 0.04562224]
4
Robbery_Burglary
1/1 [==============================] - 0s 202ms/step
[0.19643831 0.13988473 0.12358802 0.14328463 0.3111303  0.03655894
 0.049115  ]
[0.2081141  0.14046428 0.12589309 0.13979162 0.30047664 0.03958989
 0.04567041]
4
Robbery_Burglary
1/1 [==============================] - 0s 194ms/step
[0.19674788 0.13988677 0.12348843 0.14321768 0.31095496 0.03654796
 0.04915633]
[0.20807973 0.14045891 0.12586388 0.13992918 0.30042878 0.03952526
 0.04571431]
4
Robbery_Burglary
1/1 [==============================] - 0s 159ms/step
[0.19650537 0.14007472 0.12352409 0.14211176 0.31216606 0.03670089
 0.04891703]
[0.2080588  0.14043872 0.12580255 0.14004704 0.30045462 0.03945774
 0.04574049]
4
Robbery_Burglary
1/1 [==============================] - 0s 189ms/step
[0.19170256 0.14355

...

1/1 [==============================] - 0s 144ms/step
[0.6922121  0.12021737 0.00428214 0.03404204 0.04596204 0.09434696
 0.00893733]
[0.34433493 0.13819943 0.09006278 0.10580606 0.23959388 0.04559296
 0.03641009]
0
video rec start
Arson
1/1 [==============================] - 0s 136ms/step
[0.5923227  0.09939833 0.00662511 0.08972872 0.08283198 0.12206071
 0.00703245]
[0.34737158 0.13779953 0.08907199 0.10539719 0.23799409 0.04624621
 0.03611951]
0
video rec start
Arson
1/1 [==============================] - 0s 126ms/step
[0.33960667 0.17187145 0.04799778 0.11731966 0.14741842 0.16025394
 0.0155321 ]
[0.34843376 0.13796659 0.08840201 0.10520466 0.23689939 0.0471982
 0.03589555]
0
video rec start
Arson
1/1 [==============================] - 0s 119ms/step
[0.3417051  0.17107199 0.04825387 0.11703446 0.14562553 0.16087955
 0.0154295 ]
[0.34959787 0.13814268 0.08784804 0.10508678 0.23555295 0.04813209
 0.03563975]
0
video rec start
Arson
1/1 [==============================] - 0s 124ms/step


1/1 [==============================] - 0s 112ms/step
[0.24112684 0.12719181 0.09723003 0.13066971 0.22334236 0.16222887
 0.01821032]
[0.3670247  0.13847502 0.07496501 0.10320131 0.2041084  0.08527625
 0.02694941]
0
video rec start
Arson
1/1 [==============================] - 0s 121ms/step
[0.23918423 0.12802987 0.09754739 0.1306293  0.22307496 0.16329172
 0.01824252]
[0.36727577 0.13842377 0.074682   0.10314887 0.20344774 0.0862927
 0.02672925]
0
video rec start
Arson
1/1 [==============================] - 0s 126ms/step
[0.263637   0.1185788  0.10767813 0.12050739 0.20485511 0.16710865
 0.01763493]
[0.36770847 0.13826762 0.07450219 0.10301635 0.20266226 0.08733927
 0.0265039 ]
0
video rec start
Arson
1/1 [==============================] - 0s 120ms/step
[0.2576614  0.12372439 0.107481   0.11532162 0.20930125 0.16999578
 0.01651459]
[0.3681018  0.13815336 0.07432178 0.10284373 0.20190349 0.08840795
 0.02626795]
0
video rec start
Arson
1/1 [==============================] - 0s 122ms/step


1/1 [==============================] - 0s 147ms/step
[0.17223683 0.1649375  0.05436616 0.19464546 0.2173573  0.18335807
 0.01309868]
[0.377935   0.13992584 0.06103146 0.10262524 0.17660406 0.12360536
 0.01827309]
0
video rec start
Arson
1/1 [==============================] - 0s 135ms/step
[0.16999596 0.1716796  0.05521301 0.19554704 0.21683592 0.17796981
 0.01275859]
[0.37782425 0.14029847 0.0604388  0.10298581 0.1757035  0.12473365
 0.01801548]
0
video rec start
Arson
1/1 [==============================] - 0s 136ms/step
[0.17055447 0.17228816 0.05489215 0.19465032 0.2167016  0.17823239
 0.01268094]
[0.37772322 0.1406466  0.05985443 0.10338534 0.17478782 0.1258563
 0.01774624]
0
video rec start
Arson
1/1 [==============================] - 0s 146ms/step
[0.15461348 0.14426346 0.06718501 0.15879382 0.23430075 0.22730418
 0.01353919]
[0.3775042  0.14078099 0.05934608 0.10350381 0.1740142  0.12736335
 0.01748735]
0
video rec start
Arson
1/1 [==============================] - 0s 140ms/step


1/1 [==============================] - 0s 161ms/step
[0.16178639 0.1562482  0.04545781 0.16910474 0.2874247  0.16979858
 0.0101796 ]
[0.31699675 0.1348296  0.05977073 0.1212762  0.19149649 0.16287223
 0.01275792]
0
video rec start
Arson
1/1 [==============================] - 0s 157ms/step
[0.14885148 0.13927095 0.05183388 0.19850433 0.2937315  0.15970045
 0.00810746]
[0.3125688  0.13495101 0.06007976 0.12265352 0.19335684 0.16365296
 0.01273698]
0
video rec start
Arson
1/1 [==============================] - 0s 151ms/step
[0.18656509 0.16700765 0.09072465 0.21652746 0.18874595 0.13635357
 0.0140757 ]
[0.3084036  0.13529953 0.06069287 0.12417486 0.19440572 0.1642598
 0.01276348]
0
video rec start
Arson
1/1 [==============================] - 0s 147ms/step
[0.18375613 0.16999082 0.09109858 0.21783794 0.18804537 0.13533719
 0.0139339 ]
[0.3041353  0.13566294 0.06130992 0.1257138  0.1955022  0.164883
 0.0127927 ]
0
video rec start
Arson
1/1 [==============================] - 0s 141ms/step
[0

{'ip': '103.23.239.71', 'city': 'Anūpgarh', 'region': 'Rajasthan', 'country': 'IN', 'loc': '29.1911,73.2086', 'org': 'AS134928 Spiderlink Networks Pvt Ltd', 'postal': '335704', 'timezone': 'Asia/Kolkata', 'readme': 'https://ipinfo.io/missingauth'}
login success
sending mail to{email_to}
mail sent
Sending video to cloud
video sent to cloud
Cleaning up...
